# Optuna Study Visualization

Visualize FM hyperparameter optimization results from Optuna studies.

In [ ]:
import optuna
from pathlib import Path

In [ ]:
# Load study - update path and name as needed
study_db = "results/optuna_fm_gambling.db"
study_name = "fm_gambling"  # Update with actual study name

study = optuna.load_study(
    study_name=study_name,
    storage=f"sqlite:///{study_db}"
)

## Study Summary

In [ ]:
print(f"Best trial: {study.best_trial.number}")
print(f"Best value (neg RMSE): {study.best_value:.4f}")
print(f"Best RMSE: {-study.best_value:.4f}")
print(f"\nBest parameters:")
for key, value in study.best_params.items():
    print(f"  {key}: {value}")

In [ ]:
# Trials dataframe
study.trials_dataframe()

## Optimization History

In [ ]:
fig = optuna.visualization.plot_optimization_history(study, target_name="Neg RMSE")
fig.show()

## Parameter Importances

In [ ]:
fig = optuna.visualization.plot_param_importances(study)
fig.show()

## Contour Plots

In [ ]:
fig = optuna.visualization.plot_contour(study, params=["num_factors", "learning_rate"])
fig.show()

In [ ]:
fig = optuna.visualization.plot_contour(study, params=["num_factors", "epochs"])
fig.show()

In [ ]:
fig = optuna.visualization.plot_contour(study, params=["learning_rate", "epochs"])
fig.show()

## Slice Plots

In [ ]:
fig = optuna.visualization.plot_slice(study, params=["num_factors", "learning_rate"])
fig.show()

## Timeline

In [ ]:
fig = optuna.visualization.plot_timeline(study)
fig.show()

## Parallel Coordinate

In [ ]:
fig = optuna.visualization.plot_parallel_coordinate(study)
fig.show()